In [14]:
import requests
import json
import pandas as pd
from datetime import datetime
from time import sleep

In [15]:
# cidades = pd.read_excel("cidades.xlsx", converters={'Codigo':str})
# cidades.Codigo = cidades.Codigo.str.zfill(5)
# cidades.UF = cidades.UF.str.lower()
# cidades.head()


In [16]:
cidades = pd.read_excel("cidades_base_tse.xlsx",converters={"CD_MUNICIPIO":str}).drop("NM_UE",axis=1)
cidades.CD_MUNICIPIO = cidades.CD_MUNICIPIO.str.zfill(5)
cidades.SG_UF = cidades.SG_UF.str.lower()
cidades.columns = ['UF','Codigo','Municipio']
cidades.head()

,UF,Codigo,Municipio
0,ac,01007,BUJARI
1,ac,01015,CAPIXABA
2,ac,01074,CRUZEIRO DO SUL
3,ac,01570,ASSIS BRASIL
4,ac,01040,MARECHAL THAUMATURGO


In [17]:
# escolha do municipio
estado = 'sp'
cidade = '71072'

In [18]:
# arquivo tipo municipio

df2 = pd.DataFrame()

for i,city in cidades.iterrows():
    estado = city['UF']
    cidade = city['Codigo']
    try:
        # print(cidade)
        data = requests.get(
        'https://resultados.tse.jus.br/oficial/ele2022/545/dados/'+estado+'/'+estado+str(cidade)+'-c0001-e000545-v.json'
        )
    
        json_data = json.loads(data.content)
        timestamp = datetime.now().strftime('%d_%m_%Y-%Hh%Mm%Ss')   

        # ja salva o json
        arquivo = open('BaseJsonCidades.txt','a')
        arquivo.write("Arquivo gerado as "+ timestamp + "\n")
        arquivo.write(str(json_data) + "\n")
        # print("Operação concluída no arquivo " + arquivo.name + " usando o modo de acesso " + arquivo.mode)
        arquivo.close()


        lista=[]

        for dic in json_data['abr'][1]['cand']:
            if dic['seq'] in ['1','2','3','4','5']:
                lista.append(dic)


                df = pd.DataFrame(lista)
                df["timestamp"] = timestamp
                df["uf_cidade"] = estado+cidade
                df2 = pd.concat([df2,df])
                # df2 = df2.drop_duplicates()
    except: 
        print(estado+" "+cidade+" nao encontrado")
    sleep(0.1)
    
    


In [19]:
df2

,seq,n,vap,pvap,e,st,timestamp,uf_cidade
0,1,13,1958,"29,44",S,Eleito,06_11_2022-23h10m02s,ac01007
0,1,13,1958,"29,44",S,Eleito,06_11_2022-23h10m02s,ac01007
1,2,22,4693,"70,56",N,Não eleito,06_11_2022-23h10m02s,ac01007
0,1,13,1497,"27,42",S,Eleito,06_11_2022-23h10m04s,ac01015
0,1,13,1497,"27,42",S,Eleito,06_11_2022-23h10m04s,ac01015
...,...,...,...,...,...,...,...,...
0,1,13,5100,"33,77",S,Eleito,06_11_2022-23h45m25s,sp63495
1,2,22,10004,"66,23",N,Não eleito,06_11_2022-23h45m25s,sp63495
0,1,13,30905,"52,49",S,Eleito,06_11_2022-23h45m26s,pb19372
0,1,13,30905,"52,49",S,Eleito,06_11_2022-23h45m26s,pb19372


In [21]:
df2.to_csv("basecidade/basecidades_"+timestamp+".csv", sep=";")
df2

ImportError: cannot import name 'FilePathOrBuffer' from 'pandas._typing' (c:\Users\User\anaconda3\lib\site-packages\pandas\_typing.py)

In [ ]:
df

,seq,n,vap,pvap,e,st,timestamp,uf_cidade
0,2,13,5724,"49,40",N,,30_10_2022-21h07m15s,go94137
1,1,22,5864,"50,60",N,,30_10_2022-21h07m15s,go94137
